<h1><b>Задача по предсказанию рейтинга шоколада

<h3><b>Никнейм на Kaggle: 

# Описание задачи

In [39]:
# Даны характеристики шоколадок и по ним нужно предсказать их рейтинг.

# Описание данных
# Company - компания производитель
# Specific Bean Origin - географический регион происхождения
# REF - параметр, определяющий момент внесения записи в базу данных (чем выше значение, тем "свежее" запись)
# Review - дата публикации отзыва
# Cocoa Percent - процентное содержание какао
# Company Location - страна производитель
# Rating - экспертный рейтинг
# Bean Type - используемый сорт какао-бобов, если таковой имеется
# Broad Bean Origin - географический регион происхождения какао-бобов

# Файлы с данными
# choco_train.csv - тренировочные данные
# choco_test_new.csv - тестовые данные
# choco_sample_submission.csv - пример submission

# Импорт библиотек, константы

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
import warnings
warnings.filterwarnings("ignore")

In [42]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.one_hot import OneHotEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder

In [43]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRanker, LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
import optuna

In [44]:
import association_metrics as am
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_absolute_percentage_error as MAPE, mean_absolute_error as MAE, r2_score, mean_squared_error as MSE
from scipy import stats

from sklearn.pipeline import Pipeline

In [45]:
RANDOM_STATE = 42

# Загрузка данных

In [46]:
TRAIN = "https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/chocolate_train.csv"
TEST = "https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/chocolate_test_new.csv"
# TRAIN = 'data\chocolate-rating\chocolate_train.csv'
# TEST = 'data\chocolate-rating\chocolate_test_new.csv'

In [47]:
train_df = pd.read_csv(TRAIN)

In [48]:
test_df = pd.read_csv(TEST)

Первый вариант замены пропусков

In [49]:
# def prepareData(df):
#     df['Bean Type'] = df['Bean Type'].fillna(value='\xa0')
#     df['Broad Bean Origin'] = df['Broad Bean Origin'].fillna(value='\xa0')
#     df['Cocoa Percent'] = df['Cocoa Percent'].map(lambda x: float(x.split('%')[0]))

#     for [i, row] in df[df['Broad Bean Origin'] == '\xa0'].iterrows():
#         rating = df[(df['Bean Type'] == row['Bean Type']) & (df['Company'] == row['Company'])]['Broad Bean Origin'].where(lambda x : x != '\xa0').value_counts().index
#         if not rating.shape[0]:
#             rating = df[(df['Bean Type'] == row['Bean Type']) & (df['Cocoa Percent'] == row['Cocoa Percent'])]['Broad Bean Origin'].where(lambda x : x != '\xa0').value_counts().index
#         if not rating.shape[0]:
#             rating = df[df['Bean Type'] == row['Bean Type']]['Broad Bean Origin'].where(lambda x : x != '\xa0').value_counts().index
#         if rating.shape[0]:
#             df.loc[i, 'Broad Bean Origin'] = rating[0]

#     for [i, row] in df[df['Bean Type'] == '\xa0'].iterrows():
#         rating = df[(df['Broad Bean Origin'] == row['Broad Bean Origin']) & (df['Company'] == row['Company'])]['Bean Type'].where(lambda x : x != '\xa0').value_counts().index
#         if not rating.shape[0]:
#             rating = df[(df['Broad Bean Origin'] == row['Broad Bean Origin']) & (df['Cocoa Percent'] == row['Cocoa Percent'])]['Bean Type'].where(lambda x : x != '\xa0').value_counts().index
#         if not rating.shape[0]:
#             rating = df[df['Broad Bean Origin'] == row['Broad Bean Origin']]['Bean Type'].where(lambda x : x != '\xa0').value_counts().index
#         if rating.shape[0]:
#             df.loc[i, 'Bean Type'] = rating[0]
#     return df

# train_df = prepareData(train_df)

Второй вариант замены пропусков

In [50]:
# train_df['Bean Type'] = train_df['Bean Type'].replace('\xa0', np.NaN)
# train_df['Broad Bean Origin'] = train_df['Broad Bean Origin'].replace('\xa0', np.NaN)

In [51]:
# train_df = train_df.apply(lambda x: x.astype('category') if x.dtype == 'object' else x)
# train_df['Rating_cat'] = train_df['Rating'].astype('category')
# cramersv = am.CramersV(train_df)
# cat_cor = cramersv.fit()

In [52]:
# df_clean = train_df.copy()
# df_clean = df_clean.drop('Rating_cat', axis=1)
# df_clean = df_clean.apply(lambda x: x.astype('object') if x.dtype == 'category' else x)

In [53]:
# def select_top3_corr_cols(col):
#     select_df = cat_cor.drop('Rating_cat', axis=0).drop('Rating_cat', axis=1)[col].sort_values(ascending=False)
#     return select_df[1:4].index.to_list()

In [54]:
# def create_merged_df(col_in, col_on):
#     df_pivot = pd.pivot_table(df_clean,
#                                     index=col_on,
#                                     values=col_in,
#                                     aggfunc=lambda x: str(x.mode().values).strip("[").strip("]").split(',')[0].split("' '")[0].strip("'")) \
#                                     .reset_index()
#     merged_df = pd.merge(df_clean, df_pivot, on=col_on, how='left')
#     return merged_df

In [55]:
# def fill_row(row, col_in):
#     if row[f'{col_in}_x'] is np.NaN:
#         value = row[f'{col_in}_y']
#         if value != '':
#             return row[f'{col_in}_y']
#         return np.NaN
#     return row[f'{col_in}_x']

In [56]:
# def fill_rows(df_clean, col_in):
#     df_clean[f'{col_in}_x'] = df_clean.apply(lambda row: fill_row(row, col_in), axis=1)

#     df_clean = df_clean.drop(f'{col_in}_y', axis=1)
#     df_clean = df_clean.rename(columns={f'{col_in}_x': col_in})
#     return df_clean

In [57]:
# def cleaned_df(df_clean, col_in):
#     selected_cols = select_top3_corr_cols(col_in)
#     for col_on in selected_cols:
#         df_clean = create_merged_df(col_in, col_on)
#         df_clean = fill_rows(df_clean, col_in)
#     return df_clean

In [58]:
# col_in = 'Bean Type'
# train_df = cleaned_df(df_clean, col_in)

In [59]:
# df_clean = train_df.copy()
# col_in = 'Broad Bean Origin'
# train_df = cleaned_df(df_clean, col_in)

In [60]:
train_df['Cocoa Percent'] = train_df['Cocoa Percent'].apply(lambda x: x.rstrip(' %')).astype(np.float32)
# train_df['Bean Type'] = train_df['Bean Type'].fillna('Other')
# train_df['Broad Bean Origin'] = train_df['Broad Bean Origin'].fillna('Other')
train_df.fillna(-999, inplace=True)
test_df.fillna(-999, inplace=True)

In [61]:
X = train_df.drop(['Rating', 'Review', 'Specific Bean Origin'], axis=1)
y = train_df['Rating']

In [62]:
X.head()

,Company,REF,Cocoa Percent,Company Location,Bean Type,Broad Bean Origin
0,Willie's Cacao,457,72.0,U.K.,Trinitario,Venezuela
1,Beschle (Felchlin),508,72.0,Switzerland,,Venezuela
2,Dark Forest,1554,70.0,U.S.A.,,Tanzania
3,Brasstown aka It's Chocolate,1125,72.0,U.S.A.,Trinitario,Dominican Republic
4,Pralus,32,75.0,France,Criollo,Indonesia


In [63]:
cat_features = [i for i, col in enumerate(X.columns) if X[col].dtype.name in ['category', 'object']]
cat_features

[0, 3, 4, 5]

In [64]:
# cv_scores = cross_val_score(CatBoostRegressor(cat_features=cat_features, random_state=RANDOM_STATE, verbose=0), X, y, cv=5, scoring='r2')
# print(cv_scores)
# print(cv_scores.mean())
# [0.11414346 0.11130119 0.19455129 0.31372048 0.27579957]
# 0.20190319850804608

# [0.13572818 0.10972463 0.19719156 0.34517449 0.24845675]
# 0.20725512272400043

In [75]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)
cb = CatBoostRegressor(cat_features=cat_features, random_state=145, verbose=0)
cb.fit(X_train, y_train)

feature_importances = cb.get_feature_importance()

feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

Company: 19.618332909845073
Broad Bean Origin: 18.062835705942625
REF: 17.4297849663235
Bean Type: 15.641806511962296
Cocoa Percent: 15.315565530263784
Company Location: 13.931674375662833


In [66]:
y_pred = cb.predict(X_val)

r2_score(y_val, y_pred)

0.25277766280538494

In [67]:
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns

X_train[cat_cols] = X_train[cat_cols].astype('object')
X_val[cat_cols] = X_val[cat_cols].astype('object')

In [68]:
cat_cols

Index(['Company', 'Company Location', 'Bean Type', 'Broad Bean Origin'], dtype='object')

In [69]:
p1 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols)),
    ('scaler_', StandardScaler()),
    ('model_', LinearRegression())
    ])

p2 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols, smoothing=1)),
    ('scaler_', StandardScaler()),
    ('model_', LinearRegression())
    ])

p3 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols, smoothing=100)),
    ('scaler_', StandardScaler()),
    ('model_', LinearRegression())
    ])

p4 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols, smoothing=1)),
    ('scaler_', MinMaxScaler()),
    ('model_', LinearRegression())
    ])

p5 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols, smoothing=10)),
    ('scaler_', MinMaxScaler()),
    ('model_', LinearRegression())
    ])

p6 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols, smoothing=100)),
    ('scaler_', MinMaxScaler()),
    ('model_', LinearRegression())
    ])

p7 = Pipeline([
    ('encoder_',LeaveOneOutEncoder(cols=cat_cols)),
    ('scaler_', StandardScaler()),
    ('model_', LinearRegression())
    ])

p8 = Pipeline([
    ('encoder_',LeaveOneOutEncoder(cols=cat_cols)),
    ('scaler_', MinMaxScaler()),
    ('model_', LinearRegression())
    ])


for i, p in enumerate([p1,p2,p3,p4,p5,p6,p7,p8]):
    p.fit(X_train, y_train)
    pred = p.predict(X_val)
    print(i + 1, r2_score(y_val, pred))

1 0.10540959318604781
2 0.08546853799984666
3 0.11060151298846244
4 0.08546853799984677
5 0.1054095931860477
6 0.11060151298846277
7 0.12661445306849795
8 0.12661445306849795


In [70]:
p11 = Pipeline([
    ('encoder_',OneHotEncoder(cols=cat_cols)),
    ('model_', RandomForestRegressor(n_estimators=5, max_depth=3, random_state=42))
    ])

p12 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols, smoothing=1)),
    ('model_', RandomForestRegressor(n_estimators=124, max_depth=4, random_state=42))
    ])

p13 = Pipeline([
    ('encoder_',LeaveOneOutEncoder(cat_cols)),
    ('model_', RandomForestRegressor(n_estimators=56, max_depth=4, random_state=42))
    ])

p14 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols, smoothing=100)),
    ('model_', RandomForestRegressor(n_estimators=88, max_depth=3, random_state=42))
    ])

p15 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols, smoothing=10)),
    ('model_', RandomForestRegressor(n_estimators=6, max_depth=3, random_state=42))
    ])

p16 = Pipeline([
    ('encoder_',TargetEncoder(cols=cat_cols, smoothing=100)),
    ('model_', RandomForestRegressor(n_estimators=44, max_depth=3, random_state=42))
    ])

p17 = Pipeline([
    ('encoder_',OneHotEncoder(cols=cat_cols)),
    ('model_', RandomForestRegressor(n_estimators=55, max_depth=4, random_state=42))
    ])

p18 = Pipeline([
    ('encoder_',OneHotEncoder(cols=cat_cols)),
    ('model_', RandomForestRegressor(n_estimators=77, max_depth=5, random_state=42))
    ])

for i, p in enumerate([p11,p12,p13,p14,p15,p16,p17,p18]):
    p.fit(X_train, y_train)
    pred = p.predict(X_val)
    print(i + 1, r2_score(y_val, pred))

1 0.12851747024931615
2 0.12193775649369676
3 0.06434253544424207
4 0.07578186840582957
5 0.053397325297132814
6 0.07832739446629777
7 0.14004674072547207
8 0.1440460865824552


In [ ]:
def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 10, 1500),
        'max_depth': trial.suggest_int('depth', 2, 16),
        'verbose': 0,
        'early_stopping_rounds': 50
    }

    estimator = CatBoostRegressor(**params, cat_features=cat_features)
    estimator.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)
    return cross_val_score(estimator, X_train, y_train, cv=3, scoring='r2', n_jobs=5).mean()

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=15)
print(study.best_trial)

[I 2025-05-10 14:29:44,876] A new study created in memory with name: no-name-6e55d8cc-472f-4837-8cf6-d4d5122662f9
[I 2025-05-10 14:30:02,977] Trial 0 finished with value: 0.07928644484840293 and parameters: {'learning_rate': 0.1805553403675892, 'n_estimators': 23, 'depth': 15}. Best is trial 0 with value: 0.07928644484840293.
[I 2025-05-10 14:33:33,593] Trial 1 finished with value: 0.08338913156574461 and parameters: {'learning_rate': 0.04150193104469996, 'n_estimators': 1386, 'depth': 12}. Best is trial 1 with value: 0.08338913156574461.
[I 2025-05-10 14:33:45,586] Trial 2 finished with value: 0.15640795792850437 and parameters: {'learning_rate': 0.01153891232763319, 'n_estimators': 518, 'depth': 4}. Best is trial 2 with value: 0.15640795792850437.
[I 2025-05-10 14:34:14,027] Trial 3 finished with value: 0.13440827060529437 and parameters: {'learning_rate': 0.034337663058351495, 'n_estimators': 691, 'depth': 7}. Best is trial 2 with value: 0.15640795792850437.
[I 2025-05-10 14:34:22,7

KeyboardInterrupt: 

In [ ]:
hyper_param_model = CatBoostRegressor(**study.best_params, cat_features=cat_features, verbose=False)
hyper_param_model.fit(X_train, y_train)
new_pred = hyper_param_model.predict(X_val)
r2_score(y_val, new_pred)
# 0.22796892470738905
# 0.20834902449291492

0.20834902449291492

In [ ]:
cb_train_pred = hyper_param_model.predict(X_train)
r2_score(y_train, cb_train_pred)
# 0.3794773884333069
cb_train_pred = hyper_param_model.predict(X_train)
r2_score(y_train, cb_train_pred)
# 0.3794773884333069

0.4280032632009767

In [ ]:
hyper_param_model.score(X_train, y_train)
# 0.3794773884333069

0.4280032632009767

In [ ]:
def objective_lgbm(trial):
    max_depth = trial.suggest_int("max_depth", 2, 20)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1, log=True)
    n_estimators = trial.suggest_int("n_estimators", 10, 1000)

    score = cross_val_score(LGBMRegressor(max_depth=max_depth, learning_rate=learning_rate, n_estimators=n_estimators),
                            X_train, y_train, cv=3, scoring='r2', n_jobs=-1).mean()
    return score


study = optuna.create_study(direction="maximize")
study.optimize(objective_lgbm, n_trials=30)

In [ ]:
model = LGBMRegressor(**study.best_params)
model.fit(X_train, y_train)

pred = model.predict(X_val)

r2_score(y_val, pred)

In [ ]:
model.score(X_train, y_train)